In [3]:
import torch
import torch.optim as optim

In [4]:
import dataset
import model

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

In [6]:
data = dataset.LentaDataBank()
data.load_dataset("./data/Lenta")

print("Num tokens:", data.num_tokens())
for i in range(5):
    center_word, other_words = data.get_random_context(5)
    print(center_word, other_words)

Num tokens: 448257
таганке ['"антимиры",', '1965-м', 'поставлен', 'спектакль']
не ['тульской', 'взломан', 'действительности']
обтягивающую ['быть', 'одеты', 'футболку', 'синтетического']
л ['сыктывкаре!', 'батова']
("babel") ['mountain")', '"вавилон"']


In [1]:
# Значения использовались для тестового запуска. Кол-во контексов надо будет увеличить для реальной тренировки
NUM_NEGATIVE_SAMPLES = 10
NUM_CONTEXTS = 1000

ds = dataset.Word2VecDataset(data, NUM_NEGATIVE_SAMPLES, NUM_CONTEXTS)
ds.generate_dataset()

NameError: name 'dataset' is not defined

In [8]:
def extract_word_vectors(nn_model):
    '''
    Достает векторы слов из модели
    
    Возвращает:
        input_vectors: torch.Tensor с размерностями (num_tokens, num_dimensions)
        output_vectors: torch.Tensor с размерностями (num_tokens, num_dimensions)
    '''
    
    input_vectors = nn_model.input.weight.data.cpu().clone()
    output_vectors = nn_model.output.weight.data.cpu().clone()
    return torch.t(input_vectors), output_vectors

In [9]:
nn_model = model.Word2Vec(data.num_tokens())
nn_model.type(torch.cuda.FloatTensor)
nn_model.to(device)

# С парамертами надо будет поиграться
optimizer = optim.SGD(nn_model.parameters(), lr=50, weight_decay=0)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.333, patience=3)
train_loader = torch.utils.data.DataLoader(ds, batch_size=20)

%time loss_history, acc_history = model.train(nn_model, ds, train_loader, optimizer, num_epochs=10, scheduler=scheduler, scheduler_loss=True)

Epoch #0 - train loss: 0.697762134848841, accuracy: 0.09351333552848205


KeyboardInterrupt: 